In [1]:
!python Deep2BSDE_BS_tf2.py

Epoch 50, Loss: 15, y0: 5.308
Epoch 100, Loss: 7, y0: 6.720
Epoch 150, Loss: 11, y0: 6.903
Epoch 200, Loss: 6, y0: 6.901
Epoch 250, Loss: 6, y0: 6.694
Epoch 300, Loss: 6, y0: 6.896
Epoch 350, Loss: 9, y0: 6.786
Epoch 400, Loss: 7, y0: 6.794
Epoch 450, Loss: 7, y0: 6.795
Epoch 500, Loss: 7, y0: 6.591
Epoch 550, Loss: 10, y0: 6.847
Epoch 600, Loss: 6, y0: 6.947
Epoch 650, Loss: 8, y0: 6.872
Epoch 700, Loss: 9, y0: 6.898
Epoch 750, Loss: 5, y0: 6.644
Epoch 800, Loss: 9, y0: 6.732
Epoch 850, Loss: 8, y0: 7.113
Epoch 900, Loss: 8, y0: 6.682
Epoch 950, Loss: 11, y0: 6.567
Epoch 1000, Loss: 8, y0: 6.599


2021-09-22 13:53:23.858089: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library cudart64_110.dll
2021-09-22 13:53:25.603134: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library nvcuda.dll
2021-09-22 13:53:25.634985: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:2d:00.0 name: NVIDIA GeForce RTX 3080 Ti computeCapability: 8.6
coreClock: 1.71GHz coreCount: 80 deviceMemorySize: 12.00GiB deviceMemoryBandwidth: 849.46GiB/s
2021-09-22 13:53:25.635385: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library cudart64_110.dll
2021-09-22 13:53:25.642973: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library cublas64_11.dll
2021-09-22 13:53:25.643384: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library cublasL

In [2]:
!python Deep2BSDE_BS_tf2_v2.py
!python Deep2BSDE_4_3_tf2.py
!python Deep2BSDE_4_3_tf2_modified.py
!python Deep2BSDE_4_1_tf2.py

Epoch 20, Loss: 55, y0: 11.287
Epoch 40, Loss: 17, y0: 4.742
Epoch 60, Loss: 9, y0: 6.572
Epoch 80, Loss: 8, y0: 7.079
Epoch 100, Loss: 9, y0: 6.732
Epoch 120, Loss: 7, y0: 6.885
Epoch 140, Loss: 8, y0: 6.786
Epoch 160, Loss: 8, y0: 6.794
Epoch 180, Loss: 10, y0: 6.860
Epoch 200, Loss: 8, y0: 6.822
Epoch 220, Loss: 8, y0: 6.776
Epoch 240, Loss: 8, y0: 6.838
Epoch 260, Loss: 8, y0: 6.819
Epoch 280, Loss: 8, y0: 6.737
Epoch 300, Loss: 8, y0: 6.761
Epoch 320, Loss: 9, y0: 6.774
Epoch 340, Loss: 8, y0: 6.798
Epoch 360, Loss: 8, y0: 6.800
Epoch 380, Loss: 8, y0: 6.833
Epoch 400, Loss: 8, y0: 6.818
Epoch 420, Loss: 8, y0: 6.763
Epoch 440, Loss: 9, y0: 6.798
Epoch 460, Loss: 7, y0: 6.793
Epoch 480, Loss: 8, y0: 6.822
Epoch 500, Loss: 8, y0: 6.752
Epoch 520, Loss: 8, y0: 6.878
Epoch 540, Loss: 8, y0: 6.782
Epoch 560, Loss: 8, y0: 6.780
Epoch 580, Loss: 9, y0: 6.832
Epoch 600, Loss: 8, y0: 6.839
Epoch 620, Loss: 8, y0: 6.793
Epoch 640, Loss: 8, y0: 6.804
Epoch 660, Loss: 7, y0: 6.707
Epoch 680,

2021-09-22 13:59:06.478039: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library cudart64_110.dll
2021-09-22 13:59:08.080665: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library nvcuda.dll
2021-09-22 13:59:08.096190: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:2d:00.0 name: NVIDIA GeForce RTX 3080 Ti computeCapability: 8.6
coreClock: 1.71GHz coreCount: 80 deviceMemorySize: 12.00GiB deviceMemoryBandwidth: 849.46GiB/s
2021-09-22 13:59:08.096845: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library cudart64_110.dll
2021-09-22 13:59:08.103775: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library cublas64_11.dll
2021-09-22 13:59:08.104165: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library cublasL

step:  0  loss:  4239.212369554475  Y0:  [2.95569075]  learning rate:  1.0
step:  100  loss:  508.3374689580154  Y0:  [57.44408232]  learning rate:  1.0
step:  200  loss:  155.1756593754557  Y0:  [72.02398177]  learning rate:  0.5
step:  300  loss:  59.098757947990926  Y0:  [73.29828066]  learning rate:  0.5
step:  400  loss:  41.88211051382382  Y0:  [73.87119511]  learning rate:  0.25
step:  500  loss:  48.62471435304669  Y0:  [74.22080269]  learning rate:  0.25
running time:  64.41463494300842


2021-09-22 13:59:59.072443: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library cudart64_110.dll
2021-09-22 14:00:00.393193: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library nvcuda.dll
2021-09-22 14:00:00.408645: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:2d:00.0 name: NVIDIA GeForce RTX 3080 Ti computeCapability: 8.6
coreClock: 1.71GHz coreCount: 80 deviceMemorySize: 12.00GiB deviceMemoryBandwidth: 849.46GiB/s
2021-09-22 14:00:00.409344: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library cudart64_110.dll
2021-09-22 14:00:00.416462: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library cublas64_11.dll
2021-09-22 14:00:00.416818: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library cublasL

118
Epoch 50, Loss: 310, y0: 46.835
Epoch 100, Loss: 414, y0: 55.991
Epoch 150, Loss: 243, y0: 64.733
Epoch 200, Loss: 90, y0: 70.394
Epoch 250, Loss: 115, y0: 73.371
Epoch 300, Loss: 95, y0: 75.010
Epoch 350, Loss: 83, y0: 76.192
Epoch 400, Loss: 76, y0: 76.820
Epoch 450, Loss: 121, y0: 77.237
Epoch 500, Loss: 78, y0: 77.307
Epoch 550, Loss: 65, y0: 77.151
Epoch 600, Loss: 67, y0: 77.181
Epoch 650, Loss: 64, y0: 77.207
Epoch 700, Loss: 81, y0: 76.594
Epoch 750, Loss: 74, y0: 77.846
Epoch 800, Loss: 81, y0: 77.669
Epoch 850, Loss: 95, y0: 77.717
Epoch 900, Loss: 88, y0: 77.622
Epoch 950, Loss: 251, y0: 78.864
Epoch 1000, Loss: 331, y0: 78.603
Epoch 1050, Loss: 247, y0: 77.821
Epoch 1100, Loss: 260, y0: 78.327
Epoch 1150, Loss: 175, y0: 78.011
Epoch 1200, Loss: 189, y0: 77.862
Epoch 1250, Loss: 177, y0: 77.894
Epoch 1300, Loss: 224, y0: 78.151
Epoch 1350, Loss: 121, y0: 77.657
Epoch 1400, Loss: 177, y0: 78.082
Epoch 1450, Loss: 96, y0: 77.741
Epoch 1500, Loss: 123, y0: 77.484
Epoch 1550

2021-09-22 14:01:06.094735: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library cudart64_110.dll
2021-09-22 14:01:07.727998: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library nvcuda.dll
2021-09-22 14:01:07.744990: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:2d:00.0 name: NVIDIA GeForce RTX 3080 Ti computeCapability: 8.6
coreClock: 1.71GHz coreCount: 80 deviceMemorySize: 12.00GiB deviceMemoryBandwidth: 849.46GiB/s
2021-09-22 14:01:07.745577: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library cudart64_110.dll
2021-09-22 14:01:07.752773: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library cublas64_11.dll
2021-09-22 14:01:07.753149: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library cublasL

Epoch 9400, Loss: 230, y0: 77.736
Epoch 9450, Loss: 184, y0: 77.970
Epoch 9500, Loss: 216, y0: 78.591
Epoch 9550, Loss: 157, y0: 77.573
Epoch 9600, Loss: 15, y0: 77.508
Epoch 9650, Loss: 13, y0: 77.156
Epoch 9700, Loss: 8, y0: 77.217
Epoch 9750, Loss: 19, y0: 77.885
Epoch 9800, Loss: 9, y0: 77.359
Epoch 9850, Loss: 8, y0: 77.372
Epoch 9900, Loss: 13, y0: 77.027
Epoch 9950, Loss: 11, y0: 77.280
Epoch 10000, Loss: 94, y0: 77.528
step:  0  loss:  0.2752530616714354  Y0:  [0.52055618]  learning rate:  0.001
step:  100  loss:  0.01147728692139308  Y0:  [0.49269072]  learning rate:  0.001
step:  200  loss:  0.043754804204937145  Y0:  [0.46626177]  learning rate:  0.001
step:  300  loss:  0.028512090931128452  Y0:  [0.44377258]  learning rate:  0.001
step:  400  loss:  0.008243191038113214  Y0:  [0.4271195]  learning rate:  0.001


2021-09-22 14:04:37.373577: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library cudart64_110.dll
2021-09-22 14:04:38.687764: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library nvcuda.dll
2021-09-22 14:04:38.702252: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:2d:00.0 name: NVIDIA GeForce RTX 3080 Ti computeCapability: 8.6
coreClock: 1.71GHz coreCount: 80 deviceMemorySize: 12.00GiB deviceMemoryBandwidth: 849.46GiB/s
2021-09-22 14:04:38.702905: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library cudart64_110.dll
2021-09-22 14:04:38.709716: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library cublas64_11.dll
2021-09-22 14:04:38.710097: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library cublasL

step:  500  loss:  0.012428793170173985  Y0:  [0.41137082]  learning rate:  0.001
step:  600  loss:  0.016923198703843588  Y0:  [0.39696044]  learning rate:  0.001
step:  700  loss:  0.05101241543872511  Y0:  [0.38457257]  learning rate:  0.001
step:  800  loss:  0.0001991719859595254  Y0:  [0.37705698]  learning rate:  0.001
step:  900  loss:  0.0008344255417542188  Y0:  [0.37211994]  learning rate:  0.001
step:  1000  loss:  0.016612799204938616  Y0:  [0.36464088]  learning rate:  0.001
step:  1100  loss:  0.024886046147223152  Y0:  [0.35832971]  learning rate:  0.001
step:  1200  loss:  0.0008915859749795074  Y0:  [0.35068529]  learning rate:  0.001
step:  1300  loss:  0.0025148439282875164  Y0:  [0.34669425]  learning rate:  0.001
step:  1400  loss:  0.00162304958338771  Y0:  [0.34191422]  learning rate:  0.001
step:  1500  loss:  0.009749764352558062  Y0:  [0.33849264]  learning rate:  0.001
step:  1600  loss:  0.004114966816656544  Y0:  [0.33460508]  learning rate:  0.001
step:  